# Chapter 4: Generalising Linear Models

C3: we used a linear combo of input vars to predict the mean of output var. We assumed the mean was Gaussian distributed.

Some times this isn't appropriate. This chapter looks at those circumstances.

Sections in this chapter:
* Generalised LM
* Logistic Regression (LR), inverse link funcs
* Simple LR
* Multiple LR
* Softmax
* Multinomial LR
* Poisson Regression (PR)
* Zero inflated PR